In [12]:
import json
import numpy as np
import pandas as pd
import os
import cobra as cb
import pandas as pd

In [3]:
os.listdir()

['Microbes.tsv',
 'BasisChangeCauses.json',
 'InternalBasisDifferences.json',
 'Microbes.png',
 'Untitled.ipynb',
 'SpeciesNetworks',
 'SpcMetNetworks',
 'Metabolites.png',
 'InternalFluxes',
 'Metabolites.tsv',
 '.jovianrc',
 '.ipynb_checkpoints',
 'ExchangeFluxes',
 'BasisTimes.tsv',
 'MetaboliteNetworks']

In [4]:
with open('BasisChangeCauses.json') as fl:
    basis_causes = json.load(fl)

In [5]:
with open("InternalBasisDifferences.json") as fl:
    basis_diffs = json.load(fl)

In [6]:
basis_causes

{'0.22681007758802532': {'bc1012': {'Internal Flux Below 0': [],
   'Constraint Violated': ['D-Glucose_e0 exchange upper bound']}},
 '0.23234467922538346': {'bc1012': {'Internal Flux Below 0': ['rxn10122_c0'],
   'Constraint Violated': []}},
 '0.42022996887632025': {'bc1011': {'Internal Flux Below 0': [],
   'Constraint Violated': ['D-Glucose_e0 exchange upper bound']}},
 '0.42023022984260555': {},
 '0.42032719447985234': {'bc1002': {'Internal Flux Below 0': [],
   'Constraint Violated': ['D-Glucose_e0 exchange upper bound']}},
 '0.4216631872315654': {},
 '0.4217290933876869': {},
 '0.4217521993333771': {'bc1009': {'Internal Flux Below 0': [],
   'Constraint Violated': ['D-Glucose_e0 exchange upper bound']}},
 '0.421766699068233': {},
 '0.42216857467750885': {'bc1015': {'Internal Flux Below 0': ['rxn14419_c0',
    'rxn14418_c0'],
   'Constraint Violated': []}},
 '0.42289541238729833': {},
 '0.42291066128591037': {},
 '0.42340968962673986': {},
 '0.4234939885636084': {'bc1010': {'Intern

In [17]:
bc12mod = cb.io.read_sbml_model(os.path.join(os.path.expanduser("~"),"Documents","metabolic_networks","metconsin","Example","modelseedGems",'bc1012.gbk_genome.fbamodel.SBML',"bc1012.gbk_genome.fbamodel.xml"))

Set parameter TokenServer to value "infrateam.lanl.gov"


In [18]:
bc12mod.reactions.get_by_id('rxn10122_c0')

Reaction identifier,rxn10122_c0
Name,NADH dehydrogenase (ubiquinone-8 & 3.5 protons)_c0
Memory address,0x7f89e8ef49d0
Stoichiometry,cpd00004_c0 + 4.5 cpd00067_c0 + cpd15560_c0 --> cpd00003_c0 + 3.5 cpd00067_e0 + cpd15561_c0 NADH_c0 + 4.5 H+_c0 + Ubiquinone-8_c0 --> NAD_c0 + 3.5 H+_e0 + Ubiquinol-8_c0
GPR,
Lower bound,0.0
Upper bound,1000.0


In [20]:
basis_diffs['bc1012']

{'0.22681007758802532': {'Activated': {'Constraints': ['D-Glucose_e0 exchange upper bound'],
   'Fluxes': ['rxn08792_c0']},
  'Deactivated': {'Constraints': [], 'Fluxes': []}},
 '0.23234467922538346': {'Activated': {'Constraints': [], 'Fluxes': []},
  'Deactivated': {'Constraints': ['Fumarate_e0 exchange upper bound'],
   'Fluxes': ['rxn10122_c0']}},
 '0.42289541238729833': {'Activated': {'Constraints': [],
   'Fluxes': ['rxn09272_c0']},
  'Deactivated': {'Constraints': [], 'Fluxes': ['rxn08527_c0']}},
 '0.42291066128591037': {'Activated': {'Constraints': [],
   'Fluxes': ['rxn10113_c0']},
  'Deactivated': {'Constraints': [], 'Fluxes': ['rxn05654_c0_R']}},
 '0.42340968962673986': {'Activated': {'Constraints': [],
   'Fluxes': ['rxn08287_c0']},
  'Deactivated': {'Constraints': [], 'Fluxes': ['rxn14419_c0']}},
 '0.4235763066203584': {'Activated': {'Constraints': [],
   'Fluxes': ['rxn08527_c0_R']},
  'Deactivated': {'Constraints': [], 'Fluxes': ['rxn08792_c0']}},
 '0.42357634908135755': 

In [21]:
bc12mod.reactions.get_by_id('rxn08792_c0')

Reaction identifier,rxn08792_c0
Name,lactate oxidation_c0
Memory address,0x7f89e8f0caf0
Stoichiometry,cpd00159_c0 + cpd15560_c0 --> cpd00020_c0 + cpd15561_c0 L-Lactate_c0 + Ubiquinone-8_c0 --> Pyruvate_c0 + Ubiquinol-8_c0
GPR,AOLJGMBJ_04287
Lower bound,0.0
Upper bound,1000.0


In [30]:
os.listdir(os.path.join('SpcMetNetworks',"Combined"))

['SpcMetNetworksNodesCombined.tsv',
 'SpcMetNetworksEdgesSummaryCombined.tsv',
 'SpcMetNetworksEdgesCombined.tsv']

In [32]:
network_diff = pd.read_csv(os.path.join('SpcMetNetworks','Difference','SpcMetNetworksEdgesSummaryDifference.tsv'),sep='\t',index_col = 0)

In [33]:
network_comb = pd.read_csv(os.path.join('SpcMetNetworks','Combined','SpcMetNetworksEdgesSummaryCombined.tsv'),sep='\t',index_col = 0)

In [37]:
network_comb_num = network_comb[[col for col in network_comb.columns if col not in ["Source","Target"]]]

In [52]:
def make_diff_df(df):

    '''
    Makes the ``Difference`` networks table.

    :param df: DataFrame indexed by edges in the network set, with a column for each time interval
    :type df: pd.DataFrame

    :return: DataFrame indexed by edges in the network set, with a column for each transition. Values are old network edge weight minus new.
    :rtype: pd.DataFrame
    
    '''

    ddf = pd.DataFrame(index = df.index)
    for i in range(df.shape[1]-1):
        c1 = df.columns[i]
        c2 = df.columns[i+1]
        col = c1.split("-")[1]

        ddf[col] = df[c1]-df[c2]

    ddf["Source"] = [i.split("##")[0] for i in ddf.index]
    ddf["Target"] = [i.split("##")[1] for i in ddf.index]
    return ddf

In [54]:
diffdf = make_diff_df(network_comb_num)

In [55]:
diffdf.columns

Index(['0.2268', '0.4202', '0.4203', '0.4217', '0.4218', '0.4222', '0.4229',
       '0.4234', '0.4235', '0.4236', '1.1763', 'Source', 'Target'],
      dtype='object')

In [57]:
diffdf.loc[diffdf['0.2268'].abs() > 10**-3,'0.2268']

bc1012##L-Leucine##Microbe                               -0.004943
bc1012##L-Threonine##Microbe                             -0.002779
bc1012##H+##Microbe                                      -0.407939
bc1012##Acetate##Microbe                                 -0.398493
bc1012##L-Glutamate##Microbe                             -0.003362
bc1012##L-Lysine##Microbe                                -0.003758
bc1012##L-Tyrosine##Microbe                              -0.001589
bc1012##D-Glucose##Microbe                                0.190290
O2##bc1012##Metabolite                                    0.109537
bc1012##L-Arginine##Microbe                              -0.003248
bc1012##GTP##Microbe                                     -0.001783
bc1012##H2O##Microbe                                      0.476542
bc1012##AMP##Microbe                                     -0.001414
bc1012##L-Aspartate##Microbe                             -0.002644
bc1012##CMP##Microbe                                     -0.00

In [64]:
diffdf['0.2268'].loc[diffdf['0.2268'].abs().sort_values(ascending = False).index].head(10)

bc1012##Succinate##Microbe      -0.777977
bc1012##Fumarate##Microbe        0.777977
bc1012##H2O##Microbe             0.476542
bc1012##H+##Microbe             -0.407939
bc1012##Acetate##Microbe        -0.398493
bc1012##CO2##Microbe            -0.380336
bc1012##D-Glucose##Microbe       0.190290
D-Glucose##bc1012##Metabolite   -0.130369
O2##bc1012##Metabolite           0.109537
Fumarate##bc1012##Metabolite     0.033998
Name: 0.2268, dtype: float64

In [65]:
network_comb.loc[diffdf['0.2268'].abs().sort_values(ascending = False).index].head(10)

,0.0000-0.2268,0.2323-0.4202,0.4202-0.4202,0.4202-0.4203,0.4203-0.4217,0.4217-0.4217,0.4217-0.4218,0.4218-0.4222,0.4222-0.4229,0.4229-0.4234,0.4234-0.4235,0.4235-0.4236,0.4242-1.1763,1.1763-2.5000,Source,Target
bc1012##Succinate##Microbe,1.000000,1.777977,1.777977,1.777977,1.777977,1.777977,1.777977,1.777977,1.777977,0.000000,0.000000,0.000000,0.000000,0.000000,bc1012,Succinate
bc1012##Fumarate##Microbe,-1.000000,-1.777977,-1.777977,-1.777977,-1.777977,-1.777977,-1.777977,-1.777977,-1.777977,0.000000,0.000000,0.000000,0.000000,0.000000,bc1012,Fumarate
bc1012##H2O##Microbe,1.743445,1.266903,1.266903,1.266903,1.266903,1.266903,1.266903,1.266903,1.266903,11.412337,11.412337,11.412337,3.030734,3.030734,bc1012,H2O
bc1012##H+##Microbe,1.230862,1.638801,1.638801,1.638801,1.638801,1.638801,1.638801,1.638801,1.638801,-1.253705,-1.253705,-1.253705,1.643691,1.643691,bc1012,H+
bc1012##Acetate##Microbe,1.365017,1.763510,1.763510,1.763510,1.763510,1.763510,1.763510,1.763510,1.763510,-0.130316,-0.130316,-0.130316,1.766712,1.766712,bc1012,Acetate
bc1012##CO2##Microbe,1.622887,2.003222,2.003222,2.003222,2.003222,2.003222,2.003222,2.003222,2.003222,2.029026,2.029026,2.029026,2.003179,2.003179,bc1012,CO2
bc1012##D-Glucose##Microbe,-0.809710,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,bc1012,D-Glucose
D-Glucose##bc1012##Metabolite,0.000000,0.130369,0.130369,0.130369,0.130369,0.130369,0.130369,0.130369,0.130369,3.131653,3.131653,3.131653,0.171472,0.171472,D-Glucose,bc1012
O2##bc1012##Metabolite,0.152993,0.043456,0.043456,0.043456,0.043456,0.043456,0.043456,0.043456,0.043456,-1.565826,-1.565826,-1.565826,0.000000,0.000000,O2,bc1012
Fumarate##bc1012##Metabolite,0.033998,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Fumarate,bc1012
